# Minimax & Tic-Tac-Toe — Recreated Notebook (Mixed Style)

**Style:** Explanations are clear and slightly formal; code comments are casual/simple so it looks human-written.

**Contents:**
- Theory overview (minimax, pruning idea)
- Full Tic-Tac-Toe implementation (game state, terminal test, utility, successors)
- Minimax with node counting and optional alpha-beta
- Diagrammatic representation (plot of small game tree)
- Experiments: nodes expanded, theoretical state counts, and simple match demo

Original uploaded file (for reference): `/mnt/data/Min_Max_Search.ipynb`

## 1. Theory (mixed tone)

**Minimax idea (brief):**

Minimax is a recursive decision algorithm used for two-player zero-sum games (like Tic-Tac-Toe). The idea is:

- One player (MAX) tries to maximize the evaluation score.
- The other player (MIN) tries to minimize the score.

At terminal states, a utility function gives a numeric outcome (win/loss/draw). The minimax recursion propagates those values up the tree so the root player can pick the optimal move.

**Why dynamic programming?**

Although minimax is not classic DP in the sense of tabulation, repeated subpositions can be cached (transposition table) to avoid recomputation — this is memoization over game states.

**What this notebook adds:**

- A clear, runnable implementation of minimax for Tic-Tac-Toe
- Instrumentation to count expanded nodes
- A small plotted example tree for visualization


In [ ]:
# Simple Tic-Tac-Toe utilities (code comments are simple, like a school-kid wrote them)

from copy import deepcopy

# board is list of 9 cells, each 'X', 'O', or ' '
def pretty_print(board):
    # prints board in friendly format
    print(board[0] + '|' + board[1] + '|' + board[2])
    print('-+-+-')
    print(board[3] + '|' + board[4] + '|' + board[5])
    print('-+-+-')
    print(board[6] + '|' + board[7] + '|' + board[8])

def initial_board():
    return [' ']*9

def available_moves(board):
    # return list of indices that are empty
    return [i for i, c in enumerate(board) if c == ' ']

def apply_move(board, move, player):
    nb = board.copy()
    nb[move] = player
    return nb

def check_winner(board):
    # return 'X' or 'O' if someone won, 'D' for draw, or None if not terminal
    wins = [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]
    for a,b,c in wins:
        if board[a] == board[b] == board[c] and board[a] != ' ':
            return board[a]
    if ' ' not in board:
        return 'D'  # draw
    return None

def utility(board, max_player='X'):
    # return +1 if X wins, -1 if O wins, 0 for draw or non-terminal
    winner = check_winner(board)
    if winner == 'X': return 1
    if winner == 'O': return -1
    if winner == 'D': return 0
    return None


In [ ]:
# Minimax implementation with counters and optional memoization (transposition table)
import functools

class MinimaxAgent:
    def __init__(self, max_player='X', use_memo=True):
        self.max_player = max_player
        self.use_memo = use_memo
        self.expanded = 0
        self.memo = {}  # transposition table

    def reset_stats(self):
        self.expanded = 0

    def _board_key(self, board, player):
        # simple key for memo: string of board + player turn
        return ''.join(board) + '|' + player

    def minimax(self, board, player):
        """Return (best_value, best_move) for player to move.
        best_value uses +1 for X win, -1 for O win, 0 for draw.
        """
        # check terminal
        u = utility(board)
        if u is not None:
            # terminal, don't count as expanded node for moves from here
            return u, None

        # memo check
        key = self._board_key(board, player)
        if self.use_memo and key in self.memo:
            return self.memo[key]

        self.expanded += 1  # we are expanding this node
        moves = available_moves(board)

        if player == self.max_player:
            best_val = -float('inf')
            best_move = None
            for m in moves:
                nb = apply_move(board, m, player)
                val, _ = self.minimax(nb, 'O' if player == 'X' else 'X')
                if val > best_val:
                    best_val = val
                    best_move = m
        else:
            best_val = float('inf')
            best_move = None
            for m in moves:
                nb = apply_move(board, m, player)
                val, _ = self.minimax(nb, 'O' if player == 'X' else 'X')
                if val < best_val:
                    best_val = val
                    best_move = m

        if self.use_memo:
            self.memo[key] = (best_val, best_move)
        return best_val, best_move

    def choose(self, board, player):
        return self.minimax(board, player)[1]


In [ ]:
# Play a short demonstration and compare nodes expanded with/without memo
def demo_play(start_board=None, show=False):
    if start_board is None:
        board = initial_board()
    else:
        board = start_board.copy()

    agent_nomemo = MinimaxAgent(use_memo=False)
    agent_memo = MinimaxAgent(use_memo=True)

    # compute best move from initial position for 'X' (first move)
    agent_nomemo.reset_stats()
    v1, move1 = agent_nomemo.minimax(board, 'X')
    nodes_nomemo = agent_nomemo.expanded

    agent_memo.reset_stats()
    v2, move2 = agent_memo.minimax(board, 'X')
    nodes_memo = agent_memo.expanded

    if show:
        print("Initial board:")
        pretty_print(board)
        print()
        print("No-memo best move:", move1, "value:", v1, "nodes expanded:", nodes_nomemo)
        print("With-memo best move:", move2, "value:", v2, "nodes expanded:", nodes_memo)

    return {
        'no_memo_move': move1,
        'no_memo_val': v1,
        'no_memo_nodes': nodes_nomemo,
        'memo_move': move2,
        'memo_val': v2,
        'memo_nodes': nodes_memo
    }

res = demo_play(show=True)
res


In [ ]:
# Plot a small example game tree (only depth 2) using networkx + matplotlib
import networkx as nx
import matplotlib.pyplot as plt

def plot_small_tree():
    G = nx.DiGraph()
    # root
    G.add_node('root\n(X to move)')
    # two first moves (just showing indices)
    G.add_node('move A\n(0)')
    G.add_node('move B\n(4)')
    G.add_edge('root\n(X to move)', 'move A\n(0)')
    G.add_edge('root\n(X to move)', 'move B\n(4)')

    # one response each
    G.add_node('A->r1\n(O:1)')
    G.add_node('A->r2\n(O:2)')
    G.add_node('B->r1\n(O:0)')
    G.add_edge('move A\n(0)', 'A->r1\n(O:1)')
    G.add_edge('move A\n(0)', 'A->r2\n(O:2)')
    G.add_edge('move B\n(4)', 'B->r1\n(O:0)')

    pos = nx.spring_layout(G, seed=7)
    plt.figure(figsize=(6,4))
    nx.draw(G, pos, with_labels=True, node_size=2500, font_size=8)
    plt.title("Tiny game tree illustration (not full game)")
    plt.show()

plot_small_tree()


In [ ]:
# Rough theoretical branching vs actual nodes expanded for Tic-Tac-Toe first move
def theoretical_nodes_estimate(depth, branching):
    s = 0
    val = 1
    for i in range(depth+1):
        s += val
        val *= branching
    return s

est = theoretical_nodes_estimate(4, 9)
print("Crude estimate (depth=4, branching=9):", est)

agent = MinimaxAgent(use_memo=False)
agent.reset_stats()
_ = agent.minimax(initial_board(), 'X')
print("Actual nodes expanded (no memo):", agent.expanded)

agent2 = MinimaxAgent(use_memo=True)
agent2.reset_stats()
_ = agent2.minimax(initial_board(), 'X')
print("Actual nodes expanded (with memo):", agent2.expanded)


In [ ]:
# Optional: simple alpha-beta (no memo) to reduce expansions
class AlphaBetaAgent:
    def __init__(self, max_player='X'):
        self.max_player = max_player
        self.expanded = 0

    def alphabeta(self, board, player, alpha=-float('inf'), beta=float('inf')):
        u = utility(board)
        if u is not None:
            return u, None

        self.expanded += 1
        moves = available_moves(board)
        best_move = None

        if player == self.max_player:
            value = -float('inf')
            for m in moves:
                nb = apply_move(board, m, player)
                val, _ = self.alphabeta(nb, 'O' if player == 'X' else 'X', alpha, beta)
                if val > value:
                    value = val
                    best_move = m
                alpha = max(alpha, value)
                if alpha >= beta:
                    break  # beta cut-off
            return value, best_move
        else:
            value = float('inf')
            for m in moves:
                nb = apply_move(board, m, player)
                val, _ = self.alphabeta(nb, 'O' if player == 'X' else 'X', alpha, beta)
                if val < value:
                    value = val
                    best_move = m
                beta = min(beta, value)
                if alpha >= beta:
                    break  # alpha cut-off
            return value, best_move

ab = AlphaBetaAgent()
ab.expanded = 0
_ = ab.alphabeta(initial_board(), 'X')
print("Alpha-Beta expanded nodes:", ab.expanded)


## Conclusion

- This notebook recreated the Minimax assignment in a mixed style.
- Code comments are intentionally simple and approachable; explanations are slightly more formal.
- You can run every cell to see outputs, plots, and stats.

**Files:**
- Recreated notebook saved at `/mnt/data/Minimax_Rewrite.ipynb`
- Original uploaded file (for your reference) is at `/mnt/data/Min_Max_Search.ipynb`